In [148]:
# !pip3 install requests
# !pip3 install python-dotenv
# !pip3 install gspread
# !pip3 install oauth2client
# !pip3 install google-api-python-client

In [7]:
import os
import json
import requests
import gspread
import datetime
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from dotenv import load_dotenv

In [8]:
load_dotenv('.env')

True

# Retrieve Canvas Inbox starred messages (conversations)

In [9]:
# Replace with your Canvas API token
access_token = os.getenv('CANVAS_ACCESS_TOKEN')
base_url = os.getenv('CANVAS_BASE_URL')

# Headers for authentication
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Parameters to control pagination
params = {
    'per_page': 50  # Adjust this number as needed, up to the maximum allowed by the API
}

In [10]:
# Group members in ECS 161
member1_id = os.getenv('MEMBER1_CANVASID')
member2_id = os.getenv('MEMBER2_CANVASID')
member3_id = os.getenv('MEMBER3_CANVASID')
member4_id = os.getenv('MEMBER4_CANVASID')
member5_id = os.getenv('MEMBER5_CANVASID')
member6_id = os.getenv('MEMBER6_CANVASID')
member7_id = os.getenv('MEMBER7_CANVASID')
group_id = [member1_id, member2_id, member3_id, member4_id, member5_id, member6_id, member7_id]

In [11]:
def retrieve_feedback_sender(base_url, headers, params, data_validation=False, group_id=None):
    if group_id is None:
        group_id = []

    conversations_response = requests.get(f'{base_url}/conversations', headers=headers, params=params)
    conversations = conversations_response.json()
    print(json.dumps(conversations, indent=4))

    # List to store all messages with sender names
    sender_data = []

    # Get sender names from starred conversations and exclude group members
    for conversation in conversations:
        if conversation["starred"]:
            if data_validation:
                for participant in conversation["participants"]:
                    if str(participant["id"]) in group_id:
                        continue
                    else:
                        sender_id = participant["id"]
                        sender_name = participant["name"]
                        sender_data.append([sender_id, sender_name])
            else:
                sender_id = conversation["participants"][0]["id"]
                sender_name = conversation["participants"][0]["name"]
                sender_data.append([sender_id, sender_name])

    if data_validation:
        print("Number of starred conversations (# of senders) excluding group members: ", len(sender_data))
    else:
        print("Number of starred conversations (# of senders): ", len(sender_data))

    print(sender_data)  # Print sender data for testing

    return sender_data

In [12]:
sender_data = retrieve_feedback_sender(base_url, headers, params, data_validation=True, group_id=group_id)

[
    {
        "id": 14048425,
        "subject": "[ECS 161] Peer Feedback",
        "workflow_state": "read",
        "last_message": "Hi, thank you for your feedback!\n\nCompleteness: 5/5\nComplete. There is feedback across all catego...",
        "last_message_at": "2024-05-31T20:49:50Z",
        "last_authored_message": "1. Informativeness\na. Presentation covers a topic that is relevant to the course 4/5\nb. Presentat...",
        "last_authored_message_at": "2024-05-05T04:13:28Z",
        "message_count": 4,
        "subscribed": true,
        "private": false,
        "starred": false,
        "properties": [],
        "audience": [
            331094,
            339615,
            261117,
            265202,
            331213
        ],
        "audience_contexts": {
            "courses": {
                "897318": []
            },
            "groups": {}
        },
        "avatar_url": "https://canvas.ucdavis.edu/images/messages/avatar-group-50.png",
        "particip

# Save result into Google Sheet

In [13]:
def append_checkbox(col, total_rows, creds=None, spreadsheet=None, worksheet=None):
    service = build('sheets', 'v4', credentials=creds)
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet.id, body={
        'requests': [
            {
                'repeatCell': {
                    'cell': {
                        'dataValidation': {
                            'condition': {'type': 'BOOLEAN'},
                            'showCustomUi': True
                        }
                    },
                    'range': {
                        'sheetId': worksheet.id,
                        'startRowIndex': 1,  # Skip the header row
                        'endRowIndex': total_rows + 1,
                        'startColumnIndex': col,  # Column B
                        'endColumnIndex': col + 1
                    },
                    'fields': 'dataValidation'
                }
            }
        ]
    }).execute()

In [14]:
def change_col_width(col, wid, creds=None, worksheet=None, spreadsheet=None):
    service = build('sheets', 'v4', credentials=creds)
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet.id, body = {
        'requests': [
            {
                'updateDimensionProperties': {
                    'range': {
                        'sheetId': worksheet.id,
                        'dimension': 'COLUMNS',
                        'startIndex': col,
                        'endIndex': col + 1
                    },
                    'properties': {
                        'pixelSize': wid  # Set the desired width in pixels
                    },
                    'fields': 'pixelSize'
                }
            }
        ]
    }).execute()

In [30]:
def find_all_existing_data(sender_data, worksheet, spreadsheet):

    all_existing_senders = set()
    for sheet in spreadsheet.worksheets():
        if sheet.title != worksheet.title:
            existing_data = sheet.get_all_values()
            existing_senders = {row[0] for row in existing_data if row}  # Assuming sender ids are in the first column
            all_existing_senders.update(existing_senders)

    new_sender_data = [row for row in sender_data if str(row[0]) not in all_existing_senders]

    print("Total number of sender data: ", len(sender_data))
    print("Total number of existing senders: ", len(all_existing_senders))
    print(all_existing_senders)
    print("Number of new senders: ", len(new_sender_data))
    print(new_sender_data)

    return new_sender_data

In [31]:
def append_data(data_to_sheet, worksheet, spreadsheet, creds):

    # Write column headers if the sheet is empty
    if worksheet.get_all_values():
        worksheet.append_row(['Sender ID', 'Sender Name'])

    # Write sender data to the worksheet
    num_breaker = 0
    num_added = 0
    total_line = 0
    num_to_add = len(data_to_sheet)
    person_to_assign = 7
    breaker_row = ['******* Finished my part? *******', '']

    for sender in data_to_sheet:
        worksheet.append_row([sender[0], sender[1]])  # Writing sender data
        print("Added", sender[1])
        num_added += 1
        total_line += 1
        # Add the breaker row after every 7 rows
        # print("Num added:", num_added)
        # print("Rounded number:", round(num_to_add / person_to_assign))
        if (person_to_assign > 0) and (num_added % round(num_to_add / person_to_assign)) == 0:
            worksheet.append_row(breaker_row)
            print("Added breaker row **********")
            num_to_add -= round(num_to_add / person_to_assign)
            person_to_assign -= 1
            num_breaker += 1
            total_line += 1
            num_added = 0 # Reset the counter for breaker row

    change_col_width(1, 200, creds, worksheet, spreadsheet)

    worksheet.update_cell(1, 3, 'Replied?')
    if total_line > 1:
        append_checkbox(2, total_line, creds, spreadsheet, worksheet)

    worksheet.update_cell(1, 4, 'Meta-Feedback Message')
    change_col_width(3, 1000, creds, worksheet, spreadsheet)

    print("Total breaker row(s) added:", num_breaker)

In [32]:
# Authenticate with Google Sheets API
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         "https://www.googleapis.com/auth/spreadsheets"
         ]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Define the folder ID where you want to save the Google Sheets file
folder_id = os.getenv('GOOGLE_DRIVE_FOLDER_ID')
# Define file name for the Google Sheets file
file_name = os.getenv('GOOGLE_SHEET_FILE_NAME')

# Get the current date and time precise to the second and format it
today_date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")

In [33]:
# Check if a file with the same name already exists
file_exists = False
for file in client.list_spreadsheet_files():
    if file['name'] == file_name:
        print("Same-name Google Sheet file already exists in the folder...")
        file_exists = True
        spreadsheet = client.open(file_name)
        break

# If the file does not exist, create a new one and write sender data to it
if not file_exists:
    print(f'Creating a new Google Sheet file named "{file_name}"...')
    spreadsheet = client.create(file_name, folder_id=folder_id)

    # Rename the default worksheet ('Sheet1') with today's date
    worksheet = spreadsheet.sheet1
    worksheet.update_title(today_date)


else:
    print(f'Creating a new worksheet in "{file_name}"...')
    # Add a new worksheet with today's date as the title and write sender data to it
    worksheet = spreadsheet.add_worksheet(title=today_date, rows="100", cols="20")  # Create a new worksheet

new_data = find_all_existing_data(sender_data, worksheet, spreadsheet)
append_data(new_data, worksheet, spreadsheet, creds)


Same-name Google Sheet file already exists in the folder...
Creating a new worksheet in "Peer Meta-Feedback List"...
Total number of sender data:  51
Total number of existing senders:  43
{'336920', '294836', '330205', '232732', '263261', '262130', '331213', '260669', '261438', '262086', '336456', '299773', '263785', '266075', '226921', '331094', '257209', '297654', '336215', '289634', '261117', '372073', '374479', '337682', 'Sender ID', '338107', '262880', '264272', '337318', '299681', '330148', '337271', '331485', '259911', '******* Finished my part? *******', '292923', '258169', '257960', '297088', '265907', '264442', '295853', '374541'}
Number of new senders:  13
[[339615, 'Nayeel Imtiaz'], [258208, 'Victor Lai'], [306543, 'Humza Muhammad'], [264874, 'Andy Truong'], [304683, 'Aileen Fong'], [257526, 'Brandon Geoffre Lugtu'], [373162, 'Jaya Singh'], [258145, 'Ethan Park'], [256838, 'Jenisa Sheth'], [266870, 'William Norboe'], [296687, 'Christina Phan'], [372517, 'Esther Cheng'], [26

There is a bug with Google Drive. Even though I have deleted the Google Sheet file from the Google Drive UI, it still shows up in this programming interface. So, I have to delete them using the programming interface to ensure that the file is deleted.

In [9]:
response = client.list_spreadsheet_files()

# Iterate over each file and ask for confirmation before deletion
for file in response:
    confirmation = input(f"Are you sure you want to delete the file '{file['name']}' ({file['id']})? Type 'yes' to confirm: ")
    if confirmation.lower() == 'yes':
        print(f"Deleting file: {file['name']} ({file['id']})")
        client.del_spreadsheet(file['id'])
        print("File has been deleted.")
    else:
        print(f"Skipping deletion of file: {file['name']} ({file['id']})")

print("Process completed.")

[{'id': '17G-ssU3SaBtHHrUsl6D80GLpXk8ViJ4cItMFAbhxjI4', 'name': 'Peer Meta-Feedback List', 'createdTime': '2024-05-25T07:07:16.260Z', 'modifiedTime': '2024-05-30T03:36:12.763Z'}]
Skipping deletion of file: Peer Meta-Feedback List (17G-ssU3SaBtHHrUsl6D80GLpXk8ViJ4cItMFAbhxjI4)
Process completed.
